In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [2]:
df = pd.read_csv('train_set_100.csv',encoding='utf-8')
df['labels'] = pd.factorize(df.mahkemes)[0]
df.head()

,ictihats,sucs,labels
0,hük hükümlülük müsader tasfi yerel mahkemeç ve...,5607 sayılı kanuna muhalefet,0
1,taraf görüle dava ankar fikri hak hukuk mahkem...,undefined,1
2,taraf muhakemes bölge adli mahkemes hukuk dair...,undefined,1
3,5271 139 140 5237 53 192 any mah 08102015 2014...,undefined,1
4,mahke bölge adli mahkemes ceza daires hüküm ce...,uyuşturucu madde ticareti yapma,2


In [3]:
X = df.ictihats.astype(str)
Y = df.labels
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [4]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

In [5]:
max_words = 150000
max_len = 300
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = keras.utils.pad_sequences(sequences,maxlen=max_len)

In [6]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [7]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 300)]             0         
                                                                 
 embedding (Embedding)       (None, 300, 50)           7500000   
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [8]:
model.fit(sequences_matrix,Y_train,batch_size=64,epochs=20,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/20
279/279 [==============================] - 58s 202ms/step - loss: -446.1936 - accuracy: 0.4675 - val_loss: -1179.9918 - val_accuracy: 0.4518
Epoch 2/20
279/279 [==============================] - 61s 217ms/step - loss: -2256.1660 - accuracy: 0.4680 - val_loss: -3839.0400 - val_accuracy: 0.4518
